# MMOB Project

# import

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
data_file ='https://raw.githubusercontent.com/DunodMax/MMOB/main/Dataset.txt'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}
from collections import namedtuple

In [2]:
LPMC.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,18,4,0,0,3,3,6,5,0.0,1,...,0.171389,0.0,0.334444,0.0,0,0.196389,0.0,0.53,0.0,0.035361
1,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.0,0.062222,0.0,0,0.117222,0.0,0.41,0.0,0.097156
2,21,5,1,1,4,3,1,5,0.0,1,...,0.083889,0.0,0.293611,0.0,0,0.167778,0.0,0.46,0.0,0.243377
3,22,6,0,0,3,3,6,1,1.0,1,...,0.109167,0.0,0.133333,0.0,0,0.093889,1.5,0.23,0.0,0.204142
4,31,8,1,5,1,3,1,1,1.0,1,...,0.101111,0.0,0.038056,0.0,0,0.055000,1.5,0.14,0.0,0.075758


# Model 0

all useful columns for this model are used as variables

In [3]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')
female=Variable('female')
age=Variable('age')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [4]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [5]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions

In [6]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

In [7]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [8]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.832163,0.107574,-35.623351,0.0
asc_drive,-1.235707,0.080400,-15.369411,0.0
asc_pt,-0.528767,0.054766,-9.655006,0.0
beta_cost,-0.169780,0.013108,-12.952753,0.0
beta_time,-5.450540,0.202684,-26.891767,0.0


In [9]:
res.compileEstimationResults(all_results)

,Model0
Number of estimated parameters,5.000000
Sample size,5000.000000
Final log likelihood,-4566.446492
Akaike Information Criterion,9142.892983
Bayesian Information Criterion,9175.478949
asc_cycle,-3.832163
asc_drive,-1.235707
asc_pt,-0.528767
beta_cost,-0.169780
beta_time,-5.450540


In [10]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [11]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [12]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [13]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model1_time_specification'
all_results['Model1_time_specification'] = biogeme_model1.estimate()
results_alt_spec_time = biogeme_model1.estimate()
results_alt_spec_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.509548,0.191028,-23.606677,0.0
asc_drive,-1.885698,0.122741,-15.363167,0.0
asc_pt,-2.339705,0.125799,-18.598714,0.0
beta_cost,-0.146062,0.014850,-9.835768,0.0
beta_time_cycle,-6.121590,0.542697,-11.279944,0.0
beta_time_drive,-6.445088,0.391347,-16.468983,0.0
beta_time_pt,-3.514567,0.258208,-13.611366,0.0
beta_time_walk,-8.204864,0.364372,-22.517829,0.0


In [14]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification
Number of estimated parameters,5,8
Sample size,5000,5000
Final log likelihood,-4566.446492,-4264.834246
Akaike Information Criterion,9142.892983,8545.668491
Bayesian Information Criterion,9175.478949,8597.806037
asc_cycle,-3.832163,-4.509552
asc_drive,-1.235707,-1.885697
asc_pt,-0.528767,-2.339704
beta_cost,-0.16978,-0.146062
beta_time,-5.45054,


### Cost specification

We compare with the model where we specify the price, to be sure to have the most interesting model for the future models. (walk and cycling don't have cost, so only 2 specific parameter for cost)

In [15]:
beta_cost_drive = Beta('beta_cost_drive', 0, None, None, 0)
beta_cost_pt = Beta('beta_cost_pt', 0, None, None, 0)

New utility function

In [16]:
v_walk_model1_spec_cost = beta_time * time_walk  
v_cycle_model1_spec_cost = asc_cycle + beta_time * time_cycle 
v_pt_model1_spec_cost = asc_pt + beta_time * time_pt + beta_cost_pt * cost_pt
v_drive_model1_spec_cost = asc_drive + beta_time * time_drive + beta_cost_drive * cost_drive

In [17]:
V_model1_spec_cost = {1: v_walk_model1_spec_cost , 2: v_cycle_model1_spec_cost, 3: v_pt_model1_spec_cost, 4: v_drive_model1_spec_cost}
logprob_model1_spec_cost = models.loglogit(V_model1_spec_cost, av, travel_mode)
biogeme_model1_spec_cost = bio.BIOGEME(database, logprob_model1_spec_cost)
biogeme_model1_spec_cost.modelName = 'Model_1_cost_specifiaction'
all_results['Model1_cost_specification'] = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.766977,0.107734,-34.965548,0.000000
asc_drive,-1.137426,0.080194,-14.183498,0.000000
asc_pt,-0.831978,0.064856,-12.828129,0.000000
beta_cost_drive,-0.214644,0.021272,-10.090610,0.000000
beta_cost_pt,0.061554,0.028466,2.162411,0.030587
beta_time,-5.411681,0.202913,-26.670010,0.000000


In [18]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification
Number of estimated parameters,5,8,6
Sample size,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834246,-4509.365231
Akaike Information Criterion,9142.892983,8545.668491,9030.730462
Bayesian Information Criterion,9175.478949,8597.806037,9069.833621
asc_cycle,-3.832163,-4.509552,-3.766977
asc_drive,-1.235707,-1.885697,-1.137426
asc_pt,-0.528767,-2.339704,-0.831978
beta_cost,-0.16978,-0.146062,
beta_time,-5.45054,,-5.411681


### Comparaison

we use log-likehood ratio as the model 0 is a reduction of the model 1

In [19]:
old  = results_generic
new  = results_alt_spec_time
new.likelihood_ratio_test(old, 0.05) #level of the statistics for a level of significance of 5%?

LRTuple(message='H0 can be rejected at level 5.0%', statistic=603.2244926369422, threshold=7.814727903251179)

# Model 2

In [20]:
LPMC.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent'],
      dtype='object')

In [21]:
model_base=biogeme_model1

In [22]:
#we're adding the attribute distance to our model 
#we're adding the interaction between age and distance
pt_interchanges=Variable('pt_interchanges')
driving_license=Variable('driving_license')

In [23]:
#defining associated coefficients
beta_pt_inter= Beta('beta_pt_inter', 0, None, None, 0)
beta_license= Beta('beta_license', 0, None, None, 0)

In [24]:
#the specification, we only consider the alternative distance for walking
#the interaction distance age is considered just for public trasnport
v_walk_model2 = v_walk_model1
v_cycle_model2 = v_cycle_model1
v_pt_model2 = v_pt_model1 + beta_pt_inter*pt_interchanges
v_drive_model2 = v_drive_model1 + asc_drive*beta_license*driving_license

In [25]:
#the estimation results
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model_2'] = biogeme_model2.estimate()
results_model2 = biogeme_model2.estimate()
results_model2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.566436,0.196943,-23.186558,0.000000
asc_drive,-2.782366,0.138488,-20.090999,0.000000
asc_pt,-2.421716,0.131400,-18.430173,0.000000
beta_cost,-0.140454,0.013889,-10.112643,0.000000
beta_license,-0.517660,0.027952,-18.519462,0.000000
beta_pt_inter,-0.107566,0.089736,-1.198700,0.230645
beta_time_cycle,-6.305814,0.559561,-11.269225,0.000000
beta_time_drive,-7.003904,0.411154,-17.034726,0.000000
beta_time_pt,-3.541164,0.282515,-12.534409,0.000000
beta_time_walk,-8.375885,0.377017,-22.216229,0.000000


In [26]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2
Number of estimated parameters,5,8,6,10
Sample size,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834246,-4509.365231,-4039.286547
Akaike Information Criterion,9142.892983,8545.668491,9030.730462,8098.573094
Bayesian Information Criterion,9175.478949,8597.806037,9069.833621,8163.745026
asc_cycle,-3.832163,-4.509552,-3.766977,-4.56644
asc_drive,-1.235707,-1.885697,-1.137426,-2.782371
asc_pt,-0.528767,-2.339704,-0.831978,-2.42172
beta_cost,-0.16978,-0.146062,,-0.140455
beta_time,-5.45054,,-5.411681,


In [27]:
#we test base_model against model_2 where base_model is a restricted version of model_2 suing the looglikehood
#ratio test

In [28]:
#we test them using the null hypothesis that our base model is the true model

In [29]:
results_base=model_base.estimate()

In [30]:
results_model2.likelihood_ratio_test(results_base, 0.05) #level of the statistics for a level of significance of 5%?

LRTuple(message='H0 can be rejected at level 5.0%', statistic=451.0953965490635, threshold=5.991464547107979)

In [31]:
#then depending on if the value I obtained was higher or lower than a certain number, we reject or accept the null hypothesis

In [32]:
from scipy.stats import chi2
threshold = chi2.ppf(.95, 2)
threshold #just in case we might want to compare to this threshold

5.991464547107979

# Model 3

We will conduct a non-linear transformation of time

In [33]:
model_base=biogeme_model2
log_time_walk = database.DefineVariable('log_time_walk', log(time_walk))
log_time_cycle = database.DefineVariable('log_time_cycle', log(time_cycle))
log_time_pt = database.DefineVariable('log_time_pt', log(time_pt))
log_time_drive = database.DefineVariable('log_time_drive', log(time_drive))
log_beta_time_drive = Beta('log_beta_time_drive', 0, None, None, 0)
log_beta_time_pt = Beta('log_beta_time_pt', 0, None, None, 0)
log_beta_time_walk = Beta('log_beta_time_walk', 0, None, None, 0)
log_beta_time_cycle = Beta('log_beta_time_cycle', 0, None, None, 0)

In [34]:
v_walk_model3 = log_beta_time_walk * log_time_walk
v_cycle_model3 = asc_cycle + log_beta_time_cycle * log_time_cycle
v_pt_model3 = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model3 = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [35]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.440356,0.227036,-10.748771,0.000000
asc_drive,-0.680104,0.180008,-3.778184,0.000158
asc_pt,1.677853,0.147456,11.378652,0.000000
beta_cost,-0.137105,0.013208,-10.380838,0.000000
beta_license,-2.101084,0.537274,-3.910635,0.000092
beta_pt_inter,-0.093153,0.073977,-1.259214,0.207953
log_beta_time_cycle,-2.464649,0.131859,-18.691557,0.000000
log_beta_time_drive,-2.344192,0.101472,-23.101945,0.000000
log_beta_time_pt,-2.025236,0.120497,-16.807408,0.000000
log_beta_time_walk,-4.598998,0.133120,-34.547664,0.000000


In [36]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3
Number of estimated parameters,5,8,6,10,10
Sample size,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834246,-4509.365231,-4039.286547,-3975.362089
Akaike Information Criterion,9142.892983,8545.668491,9030.730462,8098.573094,7970.724177
Bayesian Information Criterion,9175.478949,8597.806037,9069.833621,8163.745026,8035.896109
asc_cycle,-3.832163,-4.509552,-3.766977,-4.56644,-2.440356
asc_drive,-1.235707,-1.885697,-1.137426,-2.782371,-0.680104
asc_pt,-0.528767,-2.339704,-0.831978,-2.42172,1.677853
beta_cost,-0.16978,-0.146062,,-0.140455,-0.137105
beta_time,-5.45054,,-5.411681,,


We will compare Model_2 and Model_3 with a cox test, thus we will create a generic model

In [37]:
v_walk_model_generic = log_beta_time_walk * log_time_walk + beta_time_walk * time_walk
v_cycle_model_generic = asc_cycle + log_beta_time_cycle * log_time_cycle + beta_time_cycle * time_cycle
v_pt_model_generic = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges + beta_time_pt * time_pt
v_drive_model_generic = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license + beta_time_drive * time_drive

In [38]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.098667,0.669993,-3.132373,1.733995e-03
asc_drive,-0.319943,0.340813,-0.938763,3.478523e-01
asc_pt,2.493232,0.456387,5.462976,4.682187e-08
beta_cost,-0.129696,0.013880,-9.344263,0.000000e+00
beta_license,-4.509323,4.801157,-0.939216,3.476199e-01
beta_pt_inter,-0.035939,0.085231,-0.421662,6.732718e-01
beta_time_cycle,-2.579712,0.793493,-3.251085,1.149655e-03
beta_time_drive,-3.072414,0.574352,-5.349355,8.826825e-08
beta_time_pt,-2.942602,0.533677,-5.513827,3.511131e-08
beta_time_walk,-1.617058,0.456941,-3.538879,4.018303e-04


In [39]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic
Number of estimated parameters,5,8,6,10,10,14
Sample size,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834246,-4509.365231,-4039.286547,-3975.362089,-3948.933026
Akaike Information Criterion,9142.892983,8545.668491,9030.730462,8098.573094,7970.724177,7925.866052
Bayesian Information Criterion,9175.478949,8597.806037,9069.833621,8163.745026,8035.896109,8017.106756
asc_cycle,-3.832163,-4.509552,-3.766977,-4.56644,-2.440356,-2.098668
asc_drive,-1.235707,-1.885697,-1.137426,-2.782371,-0.680104,-0.319943
asc_pt,-0.528767,-2.339704,-0.831978,-2.42172,1.677853,2.493233
beta_cost,-0.16978,-0.146062,,-0.140455,-0.137105,-0.129696
beta_time,-5.45054,,-5.411681,,,


In [40]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=180.70704179653967, threshold=9.487729036781154)

In [41]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=52.85812582229573, threshold=9.487729036781154)

In this case the cox-test tells us to find better models, we will thus use a box-cox transform

In [42]:
lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0)
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)
v_walk_model_boxcox = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_boxcox  = asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_boxcox  = asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model_boxcox  = asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [43]:
V_model_boxcox = {1: v_walk_model_boxcox , 2: v_cycle_model_boxcox, 3: v_pt_model_boxcox, 4: v_drive_model_boxcox}
logprob_model_boxcox = models.loglogit(V_model_boxcox, av, travel_mode)
biogeme_model_boxcox = bio.BIOGEME(database, logprob_model_boxcox)
biogeme_model_boxcox.modelName = 'Model_boxcox'
all_results['Model_boxcox'] = biogeme_model_boxcox.estimate()
results_model_boxcox = biogeme_model_boxcox.estimate()
results_model_boxcox.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.794014,0.287787,-9.708617,0.000000e+00
asc_drive,-1.325817,0.237166,-5.590244,2.267511e-08
asc_pt,1.641995,0.177913,9.229203,0.000000e+00
beta_cost,-0.129339,0.013804,-9.370031,0.000000e+00
beta_elapsed_cycle,-3.661614,0.287124,-12.752715,0.000000e+00
beta_elapsed_time_drive,-3.701041,0.275373,-13.440103,0.000000e+00
beta_elapsed_time_pt,-2.718521,0.190871,-14.242718,0.000000e+00
beta_elapsed_walk,-5.667672,0.237039,-23.910323,0.000000e+00
beta_license,-1.091703,0.190783,-5.722232,1.051338e-08
beta_pt_inter,-0.147415,0.080668,-1.827429,6.763523e-02


In [44]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox
Number of estimated parameters,5,8,6,10,10,14,11
Sample size,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834246,-4509.365231,-4039.286547,-3975.362089,-3948.933026,-3953.048852
Akaike Information Criterion,9142.892983,8545.668491,9030.730462,8098.573094,7970.724177,7925.866052,7928.097705
Bayesian Information Criterion,9175.478949,8597.806037,9069.833621,8163.745026,8035.896109,8017.106756,7999.78683
asc_cycle,-3.832163,-4.509552,-3.766977,-4.56644,-2.440356,-2.098668,-2.794014
asc_drive,-1.235707,-1.885697,-1.137426,-2.782371,-0.680104,-0.319943,-1.325817
asc_pt,-0.528767,-2.339704,-0.831978,-2.42172,1.677853,2.493233,1.641995
beta_cost,-0.16978,-0.146062,,-0.140455,-0.137105,-0.129696,-0.129339
beta_time,-5.45054,,-5.411681,,,,


In [45]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_boxcox.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=172.4753879586624, threshold=3.841458820694124)

# Model 4

In [46]:

model_base=biogeme_model_boxcox
V_nested=V_model_boxcox
#create the nests
Mu_motorized=Beta("Mu_motorized",1,0,None,0)
Mu_notmotorized=Beta("Mu_notmotorized",1,0,None,0)
motorized= Mu_motorized,[3,4]
notmotorized=Mu_notmotorized, [1,2]
nests_motor=motorized, notmotorized


In [47]:
logprob_model_nested = models.lognested(V_nested, av, nests_motor, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'Model_nested_motor'
all_results['Model_nested_motor'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_motorized,1.575910,0.087271,18.057650,0.000000e+00
Mu_notmotorized,0.930221,0.118259,7.865972,3.552714e-15
asc_cycle,-2.112745,0.245615,-8.601852,0.000000e+00
asc_drive,-0.038952,0.005607,-6.946845,3.735456e-12
asc_pt,2.128566,0.114793,18.542570,0.000000e+00
beta_cost,-0.073186,0.011017,-6.643187,3.069722e-11
beta_elapsed_cycle,-2.594793,0.232780,-11.146992,0.000000e+00
beta_elapsed_time_drive,-2.573790,0.180625,-14.249327,0.000000e+00
beta_elapsed_time_pt,-1.936291,0.129158,-14.991665,0.000000e+00
beta_elapsed_walk,-4.783298,0.259125,-18.459416,0.000000e+00


This model is rejected as Mu_notmotorized is inferior to 1. We will try a crossnested model between motored and private.

In [48]:
#define the crossnests
Mu_private=Beta('Mu_private',3,1,None,0)
Mu_motorcross=Beta('Mu_Motorcross',3,1,None,0)
Alpha_Motor=Beta('Alpha_Motor',0.5,0,1,0)
Alpha_Private=1-Alpha_Motor

alpha_motor={1:0.0,
             2:0.0,
             3:Alpha_Motor,
             4:1.0 }
alpha_private={1:1.0,
               2:1.0,
               3:Alpha_Private,
               4:0.0}

nest_motor=Mu_motorcross, alpha_motor
nest_private=Mu_private, alpha_private
cross_nests=nest_private, nest_motor

In [49]:
logprob_model_crossnested = models.logcnl_avail(V_nested, av, cross_nests, travel_mode)
logprob_model_crossnested.changeInitValues(results_model_nested.getBetaValues())
biogeme_model_crossnested = bio.BIOGEME(database, logprob_model_crossnested)
biogeme_model_crossnested.modelName = 'Model_crossnested'
results_model_crossnested = biogeme_model_crossnested.estimate()
all_results['Model_crossnested'] = results_model_crossnested
results_model_crossnested.getEstimatedParameters()

,Value,Active bound,Rob. Std err,Rob. t-test,Rob. p-value
Alpha_Motor,1.000000,1.0,0.000048,21043.445644,0.000000e+00
Mu_Motorcross,1.595669,0.0,0.089656,17.797711,0.000000e+00
Mu_private,1.000000,1.0,0.135546,7.377576,1.612044e-13
asc_cycle,-2.191109,0.0,0.254209,-8.619337,0.000000e+00
asc_drive,-0.038512,0.0,0.005577,-6.905986,4.985568e-12
asc_pt,2.096581,0.0,0.115554,18.143677,0.000000e+00
beta_cost,-0.072396,0.0,0.010913,-6.633742,3.272826e-11
beta_elapsed_cycle,-2.673665,0.0,0.233992,-11.426313,0.000000e+00
beta_elapsed_time_drive,-2.523104,0.0,0.181327,-13.914677,0.000000e+00
beta_elapsed_time_pt,-1.902662,0.0,0.129591,-14.682082,0.000000e+00


The model is also rejected as alpha is 1. Moreover we see that mu_private is 1. We try another simple nest with the use of a vehicule:

In [50]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=1,[1]
nests_vehicle=vehicle, not_vehicle


In [51]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_vehicle,1.471310,0.073351,20.058348,0.000000e+00
asc_cycle,-0.869133,0.188887,-4.601336,4.197890e-06
asc_drive,-0.037452,0.008526,-4.392533,1.120378e-05
asc_pt,2.129201,0.103024,20.666969,0.000000e+00
beta_cost,-0.082863,0.011364,-7.291876,3.057554e-13
beta_elapsed_cycle,-2.475289,0.180807,-13.690208,0.000000e+00
beta_elapsed_time_drive,-2.584035,0.154796,-16.693152,0.000000e+00
beta_elapsed_time_pt,-1.940494,0.112900,-17.187739,0.000000e+00
beta_elapsed_walk,-4.797876,0.216407,-22.170637,0.000000e+00
beta_license,-28.576404,6.411480,-4.457068,8.308812e-06


In [52]:
#t-test H0:true value is 1
mu = table_nestedvehicle.loc['Mu_vehicle', 'Value']
mu_stderr = table_nestedvehicle.loc['Mu_vehicle', 'Rob. Std err']
tested_value = 1
ttest = (mu - tested_value) / mu_stderr
ttest

6.4253613399849145

In [53]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox,Model_nested_motor,Model_crossnested,Model_nested_vehicle
Number of estimated parameters,5,8,6,10,10,14,11,13,14,12
Sample size,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834246,-4509.365231,-4039.286547,-3975.362089,-3948.933026,-3953.048852,-3927.315463,-3927.440999,-3939.726438
Akaike Information Criterion,9142.892983,8545.668491,9030.730462,8098.573094,7970.724177,7925.866052,7928.097705,7880.630926,7882.881998,7903.452875
Bayesian Information Criterion,9175.478949,8597.806037,9069.833621,8163.745026,8035.896109,8017.106756,7999.78683,7965.354438,7974.122703,7981.659194
asc_cycle,-3.832163,-4.509552,-3.766977,-4.56644,-2.440356,-2.098668,-2.794014,-2.112745,-2.191109,-0.869133
asc_drive,-1.235707,-1.885697,-1.137426,-2.782371,-0.680104,-0.319943,-1.325817,-0.038952,-0.038512,-0.037452
asc_pt,-0.528767,-2.339704,-0.831978,-2.42172,1.677853,2.493233,1.641995,2.128566,2.096581,2.129201
beta_cost,-0.16978,-0.146062,,-0.140455,-0.137105,-0.129696,-0.129339,-0.073186,-0.072396,-0.082863
beta_time,-5.45054,,-5.411681,,,,,,,


In [54]:
results_base=model_base.estimate()
results_model_nestedvehicle.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=26.64482951921309, threshold=3.841458820694124)

In [55]:
V_pref=V_nested
nest_pref=nests_vehicle
model_name_pref='Model_nested_vehicle'
result_pref=results_model_nestedvehicle
estimation_pref=table_nestedvehicle

### Market share

In [56]:
LPMC.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent,log_time_walk,log_time_cycle,log_time_pt,log_time_drive
0,18,4,0,0,3,3,6,5,0.0,1,...,0,0.196389,0.0,0.53,0.0,0.035361,-0.271153,-1.462456,-0.681548,-1.627658
1,20,5,1,0,4,3,1,5,0.0,1,...,0,0.117222,0.0,0.41,0.0,0.097156,-0.513607,-1.696449,-0.812181,-2.143684
2,21,5,1,1,4,3,1,5,0.0,1,...,0,0.167778,0.0,0.46,0.0,0.243377,-0.505746,-1.586101,-0.974185,-1.785115
3,22,6,0,0,3,3,6,1,1.0,1,...,0,0.093889,1.5,0.23,0.0,0.204142,-1.072295,-2.061820,-1.416754,-2.365643
4,31,8,1,5,1,3,1,1,1.0,1,...,0,0.055000,1.5,0.14,0.0,0.075758,-1.749339,-2.905485,-1.972083,-2.900422


In [57]:
LPMC[(LPMC["household_id"]==5) & (LPMC["person_n"]==1)].shape[0]

2

Some individuals make several trips. To account for this in the computation of the weights, we'll consider an individual making n trips as n indivudals. 

In [58]:
census = {
    'male_41_more':  1633263,
    'male_40_less':  2676249,
    'female_41_more':   1765143,
    'female_40_less':  2599058,
}

In [59]:
total = sum(census.values())
total

8673713

In [60]:
filters = {
    'male_41_more': (LPMC.age >= 41) & (LPMC.female == 0),
    'male_40_less': (LPMC.age <= 40) & (LPMC.female == 0),
    'female_41_more': (LPMC.age >= 41) & (LPMC.female == 1),
    'female_40_less': (LPMC.age <= 40) & (LPMC.female == 1),
}

In [61]:
sample_segments = {
    k: v.sum() for k, v in filters.items()
}
sample_segments

{'male_41_more': 1149,
 'male_40_less': 1275,
 'female_41_more': 1196,
 'female_40_less': 1380}

In [62]:
total_sample = sum(sample_segments.values())
total_sample

5000

In [63]:
weights = {
    k: census[k] * total_sample / (v * total) 
    for k, v in sample_segments.items()
}
weights

{'male_41_more': 0.8194096069112643,
 'male_40_less': 1.2099886308951033,
 'female_41_more': 0.8507729467060965,
 'female_40_less': 1.085680009425514}

In [64]:
for k, f in filters.items():
    LPMC.loc[f, 'weights'] = weights[k] 

In [65]:
#we check the weights sum to the sample size
sum_weights = LPMC['weights'].sum()
sum_weights

5000.0

# Market Shares


We will there use the market_share function defined in lab session 03-indicators, but we adapted it to our models


Here we DO NOT simulate weights ? is it ok ?

## Market Shares with Modelpref

In [66]:
female=Variable('female')
age=Variable('age')
weights=Variable('weights')

In [67]:
type(weights)
IndicatorTuple = namedtuple('IndicatorTuple', 'value lower upper') 

In [68]:
def market_share(utility):
    """Calculate the market shares of all alternatives, given the
    specification of the utility functions.

    :param utility: specification of the utility functions. It is a
        dict where the keys are the IDs of the alternatives, and the
        values are the expressions of the utility functions.
    :type utility: dict(int: biogeme.expressions.Expression)

    :return: a dictionary where each entry corresponds to an
        alternative, and associates its name with the InticatorTuple
        containing the value of the market share, and the lower and
        upper bounds of the 90% confidence interval.
    :rtype: dict(str: IndicatorTuple)

    """
    prob_WALK = models.lognested(V_pref, av, nests_vehicle, 1)
    prob_CYCLE = models.lognested(V_pref, av, nests_vehicle, 2)
    prob_PT = models.lognested(V_pref, av, nests_vehicle, 3)
    prob_DRIVE = models.lognested(V_pref, av, nests_vehicle, 4)
    """
    prob_WALK = models.loglogit(utility, None, 1)
    prob_CYCLE= models.loglogit(utility, None, 2)
    prob_PT = models.loglogit(utility, None, 3)
    prob_DRIVE = models.loglogit(utility, None, 4)
    """

    simulate = {
        'Prob. walk': prob_WALK,
        'Prob. cycle': prob_CYCLE,
        'Prob. PT': prob_PT,
        'Prob. drive': prob_DRIVE,
        'weights' : weights
        
    }
    biosim = bio.BIOGEME(database, simulate)
    simulated_values = biosim.simulate(results_model_nestedvehicle.getBetaValues())

    # We also calculate confidence intervals for the calculated quantities

    betas = biogeme_model_nestedvehicle.freeBetaNames()
    b = results_model_nestedvehicle.getBetasForSensitivityAnalysis(betas,useBootstrap=False)
    left, right = biosim.confidenceIntervals(b, 0.9)

    # Market shares are calculated using the weighted mean of the individual probabilities
    #Since lognested outputs the logarithm of the wanted probability, we'll have to compose the occurences
    #of the proba by the exponential

    ## Alternative WALK

    simulated_values['Weighted prob. walk'] = simulated_values['weights'] * np.exp(simulated_values['Prob. walk'])
    left['Weighted prob. walk'] = left['weights'] * np.exp(left['Prob. walk'])
    right['Weighted prob. walk'] = right['weights'] * np.exp(right['Prob. walk'])

    marketShare_walk = simulated_values['Weighted prob. walk'].mean()
    marketShare_walk_left = left['Weighted prob. walk'].mean()
    marketShare_walk_right = right['Weighted prob. walk'].mean()
    
    
    ## Alternative cycle

    simulated_values['Weighted prob. cycle'] =simulated_values['weights'] * np.exp(simulated_values['Prob. cycle'])
    left['Weighted prob. cycle'] = left['weights'] * np.exp(left['Prob. cycle'])
    right['Weighted prob. cycle'] = right['weights'] * np.exp(right['Prob. cycle'])

    marketShare_cycle = simulated_values['Weighted prob. cycle'].mean()
    marketShare_cycle_left = left['Weighted prob. cycle'].mean()
    marketShare_cycle_right = right['Weighted prob. cycle'].mean()

    ## Alternative pt

    simulated_values['Weighted prob. PT'] = (
        simulated_values['weights'] * np.exp(simulated_values['Prob. PT'])
    )
    left['Weighted prob. PT'] = left['weights'] * np.exp(left['Prob. PT'])
    right['Weighted prob. PT'] = right['weights'] * np.exp(right['Prob. PT'])

    marketShare_PT = simulated_values['Weighted prob. PT'].mean()
    marketShare_PT_left = left['Weighted prob. PT'].mean()
    marketShare_PT_right = right['Weighted prob. PT'].mean()
    
                                                         
    ## Alternative drive

    simulated_values['Weighted prob. drive'] = (
        simulated_values['weights'] * np.exp(simulated_values['Prob. drive'])
    )
    left['Weighted prob. drive'] = left['weights'] * np.exp(left['Prob. drive'])
    right['Weighted prob. drive'] = right['weights'] * np.exp(right['Prob. drive'])

    marketShare_drive = simulated_values['Weighted prob. drive'].mean()
    marketShare_drive_left = left['Weighted prob. drive'].mean()
    marketShare_drive_right = right['Weighted prob. drive'].mean()

    return {
            'Walk': IndicatorTuple(
                value=marketShare_walk, 
                lower=marketShare_walk_left, 
                upper=marketShare_walk_right
            ),
            'Cycle': IndicatorTuple(
                value=marketShare_cycle, 
                lower=marketShare_cycle_left, 
                upper=marketShare_cycle_right
            ),
            'Public transportation': IndicatorTuple(
                value=marketShare_PT, 
                lower=marketShare_PT_left, 
                upper=marketShare_PT_right
            ),
            'Drive': IndicatorTuple(
                value=marketShare_drive, 
                lower=marketShare_drive_left, 
                upper=marketShare_drive_right
            )
    }

In [69]:
ms = market_share(V_nested)
for k, v in ms.items():
    print(
        f'Market share for {k}: {100*v.value:.1f}% '
        f'[{100*v.lower:.1f}%, '
        f'{100*v.upper:.1f}%]'
    )

Market share for Walk: 17.3% [16.3%, 18.4%]
Market share for Cycle: 3.0% [2.5%, 3.8%]
Market share for Public transportation: 35.7% [34.0%, 39.1%]
Market share for Drive: 44.1% [40.1%, 45.8%]


In [70]:
#we check it sums up to 100
total=0
for k, v in ms.items():
    total+=100*v.value
print(total)

100.0


##  Market Shares with actual choice

In [71]:
count_walk=0
count_cycle=0
count_pt=0
count_drive=0
for c in LPMC['travel_mode']:
    if c==1:
        count_walk+=1
    if c==2:
        count_cycle+=1
    if c==3:
        count_pt+=1
    if c==4:
        count_drive+=1
print('Market share for walk ' +  str(count_walk/5000))
print('Market share for cycle ' +  str(count_cycle/5000))   
print('Market share for pt ' +  str(count_pt/5000))   
print('Market share for drive ' +  str(count_drive/5000))   

Market share for walk 0.1714
Market share for cycle 0.0294
Market share for pt 0.3498
Market share for drive 0.4494
